In [50]:
import numpy as np 
import pandas as pd

In [51]:
weather_code = pd.read_csv("weather_code.csv")

In [52]:
weather_code


,location_id,latitude,longitude,elevation,utc_offset_seconds,timezone,timezone_abbreviation
0,0,52.548330,13.407822,46,7200,Europe/Berlin,CEST
1,1,53.532513,9.980879,11,7200,Europe/Berlin,CEST
2,2,48.119507,11.550000,524,7200,Europe/Berlin,CEST
3,3,50.087870,8.653846,117,7200,Europe/Berlin,CEST
4,4,52.337433,14.500000,28,7200,Europe/Berlin,CEST


In [53]:
weather_code['Geo_location']= weather_code['latitude'].astype(str) + ',' + weather_code['longitude'].astype(str)

In [54]:
weather_code

,location_id,latitude,longitude,elevation,utc_offset_seconds,timezone,timezone_abbreviation,Geo_location
0,0,52.548330,13.407822,46,7200,Europe/Berlin,CEST,"52.54833,13.407822"
1,1,53.532513,9.980879,11,7200,Europe/Berlin,CEST,"53.532513,9.980879"
2,2,48.119507,11.550000,524,7200,Europe/Berlin,CEST,"48.119507,11.55"
3,3,50.087870,8.653846,117,7200,Europe/Berlin,CEST,"50.08787,8.653846"
4,4,52.337433,14.500000,28,7200,Europe/Berlin,CEST,"52.337433,14.5"


In [55]:
weather_code = pd.DataFrame({
    'location_id': [0,1,2,3,4],
    'latitude': [52.54833, 53.532513, 48.119507, 50.08787, 52.337433],
    'longitude': [13.407822, 9.980879, 11.55, 8.653846, 14.5],
    'elevation': [34, 45, 30, 28, 50],
    'utc_offset_seconds': [3600, 3600, 3600, 3600, 3600],
    'timezone': ['CET', 'CET', 'CET', 'CET', 'CET'],
    'timezone_abbreviation': ['CEST', 'CEST', 'CEST', 'CEST', 'CEST']
    
})

#Function to determine the place based on coordinates
def get_place(latitude, longitude):
    if latitude == 52.54833 and longitude == 13.407822:
        return 'Berlin'
    elif latitude == 53.532513 and longitude == 9.980879:
        return 'Hamburg'  # Assuming city for given coordinates
    elif latitude == 48.119507 and longitude == 11.55:
        return 'Munich'  # Assuming city for given coordinates
    elif latitude == 50.08787 and longitude == 8.653846:
        return 'Frankfurt'  # Assuming city for given coordinates
    elif latitude == 52.337433 and longitude == 14.5:
        return 'Berlin'  # Assuming city for given coordinates
    else:
        return 'Unknown'

# Add the 'place' column
weather_code['Place'] = weather_code.apply(lambda row: get_place(row['latitude'], row['longitude']), axis=1)

# Print the DataFrame to verify the changes
print(weather_code)

   location_id   latitude  longitude  elevation  utc_offset_seconds timezone  \
0            0  52.548330  13.407822         34                3600      CET   
1            1  53.532513   9.980879         45                3600      CET   
2            2  48.119507  11.550000         30                3600      CET   
3            3  50.087870   8.653846         28                3600      CET   
4            4  52.337433  14.500000         50                3600      CET   

  timezone_abbreviation      Place  
0                  CEST     Berlin  
1                  CEST    Hamburg  
2                  CEST     Munich  
3                  CEST  Frankfurt  
4                  CEST     Berlin  


In [56]:
import sqlite3
from sqlalchemy import create_engine

In [57]:
conn = sqlite3.connect('Weather.db')
cursor = conn.cursor()

In [58]:
cursor.execute('''CREATE TABLE IF NOT EXISTS weather_code
                  (location_id INTEGER PRIMARY KEY,
                   latitude FLOAT NOT NULL,
                   longitude FLOAT NOT NULL,
                   elevation INT,
                   utc_offset_seconds INT,
                   timezone VARCHAR(32),
                   timezone_abbreviation CHAR(4),
                   Place VARCHAR(24))
                ''')

In [59]:
conn.commit()

In [60]:
for index, row in weather_code.iterrows():
    cursor.execute('''INSERT INTO weather_code(
                        location_id,
                        latitude,
                        longitude,
                        elevation,
                        utc_offset_seconds,
                        timezone,
                        timezone_abbreviation,
                        
                        Place)
                      VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                   (row['location_id'],
                    row['latitude'],
                    row['longitude'],
                    row['elevation'],
                    row['utc_offset_seconds'],
                    row['timezone'],
                    row['timezone_abbreviation'],
                    row['Place']))
conn.commit()


In [61]:
cursor.execute('''SELECT * FROM weather_code''')
rows = cursor.fetchall()
for row in rows:
    print(row)
    

(0, 52.54833, 13.407822, 34, 3600, 'CET', 'CEST', 'Berlin')
(1, 53.532513, 9.980879, 45, 3600, 'CET', 'CEST', 'Hamburg')
(2, 48.119507, 11.55, 30, 3600, 'CET', 'CEST', 'Munich')
(3, 50.08787, 8.653846, 28, 3600, 'CET', 'CEST', 'Frankfurt')
(4, 52.337433, 14.5, 50, 3600, 'CET', 'CEST', 'Berlin')


In [62]:
weather = pd.read_csv("weather.csv")

In [63]:
weather

,location_id,time,temperature_2m (°C),relative_humidity_2m (%),apparent_temperature (°C),precipitation (mm),weather_code (wmo code),wind_speed_10m (km/h)
0,0,2010-01-01T00:00,-2.5,87,-7.5,0.3,73,15.8
1,0,2010-01-01T01:00,-2.6,87,-7.5,0.4,73,14.8
2,0,2010-01-01T02:00,-2.7,88,-7.7,0.5,73,16.0
3,0,2010-01-01T03:00,-2.7,88,-7.8,0.5,73,16.3
4,0,2010-01-01T04:00,-2.7,88,-7.8,0.4,73,16.3
...,...,...,...,...,...,...,...,...
631555,4,2024-05-29T19:00,21.3,49,19.9,0.0,0,9.8
631556,4,2024-05-29T20:00,19.6,59,19.0,0.0,2,7.1
631557,4,2024-05-29T21:00,16.9,73,16.2,0.0,2,8.7
631558,4,2024-05-29T22:00,16.1,78,16.1,0.9,53,4.7


In [79]:
import pandas as pd
import sqlite3

# Read the CSV file
weather = pd.read_csv("weather.csv")
print("Original columns:", weather.columns)

# Print columns to verify exact names
print("Columns before renaming:")
for col in weather.columns:
    print(f"'{col}'")

# Rename columns based on the exact names printed
weather.rename(columns={
    'temperature_2m(°C)': 'temperature_2m',
    'relative_humidity_2m(%)': 'relative_humidity_2m',
    'apparent_temperature(°C)': 'apparent_temperature',
    'precipitation(mm)': 'precipitation',
    'weather_code(wmo code)': 'weather_code',
    'wind_speed_10m(km/h)': 'wind_speed_10m'
}, inplace=True)
print("Renamed columns:", weather.columns)

# Display the first few rows to verify the content
print(weather.head())

# Connect to the SQLite database
conn = sqlite3.connect('Weather.db')
cursor = conn.cursor()

# Create the table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS weather_table
                  (location_id INTEGER PRIMARY KEY,
                   time DATETIME,
                   temperature_2m CHAR(8),
                   relative_humidity_2m DECIMAL(5, 2),
                   apparent_temperature DECIMAL(5, 2),
                   precipitation DECIMAL(5, 2),
                   weather_code INT,
                   wind_speed_10m DECIMAL(5, 2))
                ''')
conn.commit()

# Insert data into the table
for index, row in weather.iterrows():
    cursor.execute('''INSERT INTO weather_table(
                        location_id,
                        time,
                        temperature_2m,
                        relative_humidity_2m,
                        apparent_temperature,
                        precipitation,
                        weather_code,
                        wind_speed_10m)
                      VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                   (row['location_id'],
                    row['time'],
                    row['temperature_2m'],
                    row['relative_humidity_2m'],
                    row['apparent_temperature'],
                    row['precipitation'],
                    row['weather_code'],
                    row['wind_speed_10m']))
conn.commit()

# Close the connection
conn.close()


Original columns: Index(['location_id', 'time', 'temperature_2m (°C)',
       'relative_humidity_2m (%)', 'apparent_temperature (°C)',
       'precipitation (mm)', 'weather_code (wmo code)',
       'wind_speed_10m (km/h)'],
      dtype='object')
Columns before renaming:
'location_id'
'time'
'temperature_2m (°C)'
'relative_humidity_2m (%)'
'apparent_temperature (°C)'
'precipitation (mm)'
'weather_code (wmo code)'
'wind_speed_10m (km/h)'
Renamed columns: Index(['location_id', 'time', 'temperature_2m (°C)',
       'relative_humidity_2m (%)', 'apparent_temperature (°C)',
       'precipitation (mm)', 'weather_code (wmo code)',
       'wind_speed_10m (km/h)'],
      dtype='object')
   location_id              time  temperature_2m (°C)  \
0            0  2010-01-01T00:00                 -2.5   
1            0  2010-01-01T01:00                 -2.6   
2            0  2010-01-01T02:00                 -2.7   
3            0  2010-01-01T03:00                 -2.7   
4            0  2010-01-01T04:

KeyError: 'temperature_2m'

In [64]:
# Read the CSV file
weather = pd.read_csv("weather.csv")
print("Original columns:", weather.columns)

Original columns: Index(['location_id', 'time', 'temperature_2m (°C)',
       'relative_humidity_2m (%)', 'apparent_temperature (°C)',
       'precipitation (mm)', 'weather_code (wmo code)',
       'wind_speed_10m (km/h)'],
      dtype='object')


In [65]:
print(weather.columns)

Index(['location_id', 'time', 'temperature_2m (°C)',
       'relative_humidity_2m (%)', 'apparent_temperature (°C)',
       'precipitation (mm)', 'weather_code (wmo code)',
       'wind_speed_10m (km/h)'],
      dtype='object')


In [71]:
# Rename columns
weather.rename(columns={
    'location_id': 'id',
    'time': 'DateTime',
    'temperature_2m (°C)': 'temperature_2m',
    'relative_humidity_2m (%)': 'relative_humidity_2m',
    'apparent_temperature (°C)': 'apparent_temperature',
    'precipitation (mm)': 'precipitation',
    'weather_code (wmo code)': 'weather_code',
    'wind_speed_10m (km/h)': 'wind_speed_10m'
}, inplace=True)
print("Renamed columns:", weather.columns)

Renamed columns: Index(['id', 'DateTime', 'temperature_2m', 'relative_humidity_2m',
       'apparent_temperature', 'precipitation', 'weather_code',
       'wind_speed_10m'],
      dtype='object')


In [72]:
print(weather.head())


   id          DateTime  temperature_2m  relative_humidity_2m  \
0   0  2010-01-01T00:00            -2.5                    87   
1   0  2010-01-01T01:00            -2.6                    87   
2   0  2010-01-01T02:00            -2.7                    88   
3   0  2010-01-01T03:00            -2.7                    88   
4   0  2010-01-01T04:00            -2.7                    88   

   apparent_temperature  precipitation  weather_code  wind_speed_10m  
0                  -7.5            0.3            73            15.8  
1                  -7.5            0.4            73            14.8  
2                  -7.7            0.5            73            16.0  
3                  -7.8            0.5            73            16.3  
4                  -7.8            0.4            73            16.3  


In [86]:
conn = sqlite3.connect('Weather.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS weather_table
                  (location_id INTEGER PRIMARY KEY,
                   time DATETIME,
                   temperature_2m CHAR(8),
                   relative_humidity_2m DECIMAL(5, 2),
                   apparent_temperature DECIMAL(5, 2),
                   precipitation DECIMAL(5, 2),
                   weather_code INT,
                   wind_speed_10m DECIMAL(5, 2))
                ''')
conn.commit()

In [87]:
for index, row in weather.iterrows():
    print("Inserting row:", row)  # Debugging statement
    cursor.execute('''INSERT INTO weather_table(
                        location_id,
                        time,
                        temperature_2m,
                        relative_humidity_2m,
                        apparent_temperature,
                        precipitation,
                        weather_code,
                        wind_speed_10m)
                      VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',

                    (row['location_id'],
                    row['time'],
                    row['temperature_2m (°C)'],
                    row['relative_humidity_2m (%)'],
                    row['apparent_temperature (°C)'],
                    row['precipitation (mm)'],
                    row['weather_code (wmo code)'],
                    row['wind_speed_10m (km/h)'])
                    )
conn.commit()


Inserting row: location_id                                 0
time                         2010-01-01T00:00
temperature_2m (°C)                      -2.5
relative_humidity_2m (%)                   87
apparent_temperature (°C)                -7.5
precipitation (mm)                        0.3
weather_code (wmo code)                    73
wind_speed_10m (km/h)                    15.8
Name: 0, dtype: object


OperationalError: database is locked